### 主程式 llama-cli
`llama-cli` 是整個專案最基本的用法，絕大多數的指令參數都放在這裡。

#### 1. 參數說明

In [2]:
%%bash

cd llama.cpp

./build/bin/llama-cli --help # 顯示參數說明

----- common params -----

-h,    --help, --usage                  print usage and exit
--version                               show version and build info
--completion-bash                       print source-able bash completion script for llama.cpp
--verbose-prompt                        print a verbose prompt before generation (default: false)
-t,    --threads N                      number of threads to use during generation (default: -1)
                                        (env: LLAMA_ARG_THREADS)
-tb,   --threads-batch N                number of threads to use during batch and prompt processing (default:
                                        same as --threads)
-C,    --cpu-mask M                     CPU affinity mask: arbitrarily long hex. Complements cpu-range
                                        (default: "")
-Cr,   --cpu-range lo-hi                range of CPUs for affinity. Complements --cpu-mask
--cpu-strict <0|1>                      use strict CPU placement (defaul

#### 2. 首先是 `-m` 指定 gguf 格式的模型路徑，並使用 `-ngl` 指定 GPU 讀取層數：
* 建議直接設定為 `-ngl 99` 就好，就算超過 GPU 記憶體上限，系統也會自動改放到 CPU 記憶體裡面。
* 在沒有其他參數的情況下，程式會快速讀取完模型，然後自己開始瘋狂輸出，這通常是用來測試運作的速度有多快。
* 調用 GPU 時，要確認讀取訊息有出現類似以下的文字：

    ```md
    llm_load_tensors: offloading 32 repeating layers to GPU
    llm_load_tensors: offloading non-repeating layers to GPU 
    llm_load_tensors: offloaded 33/33 layers to GPU
    ```
    * 請確保最後一行寫的是 `33/33` 而不是 `30/33` 之類的，否則就代表有些模型參數沒有被放進 GPU 裡面，而是放在 CPU 裡面做運算，速度會因此大打折扣。

In [3]:
%%bash

cd llama.cpp

./build/bin/llama-cli -m llama-3-8b-inst.gguf -ngl 99

build: 1 (a3c3084) with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: llama backend init
main: load the model and apply lora adapter, if any
llama_model_loader: loaded meta data with 27 key-value pairs and 291 tensors from llama-3-8b-inst.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = 5f0b02c75b57c5855da9ae460ce51323ea669d8a
llama_model_loader: - kv   3:                           general.finetune str              = 5f0b02c75b57c5855da9ae460ce51323ea669d8a
llama_model_loader: - kv   4:                         general.size_label str              = 8.0B
llama_model_loader: - kv   5:                     


> EOF by user




#### 3. 如果想要進入互動模式，可以加上 `-if` 參數：
* 但這樣並不是 Llama 3 標準的聊天模式。

In [ ]:
%%bash

cd llama.cpp

# 請用 terminal 執行
./build/bin/llama-cli -m llama-3-8b-inst.gguf -ngl 99 -if

#### 4. 可以透過參數 `-cnv` 來啟用聊天模型，並搭配參數 `-p` 來指定系統提示：

In [ ]:
%%bash

cd llama.cpp

# 請用 terminal 執行
./build/bin/llama-cli -m llama-3-8b-inst.gguf -ngl 99 -cnv \
    -p "You are a helpful assistant. Reply in Traditional Chinese. (繁體中文)"

build: 1 (a3c3084) with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: llama backend init
main: load the model and apply lora adapter, if any


llama_model_loader: loaded meta data with 27 key-value pairs and 291 tensors from llama-3-8b-inst.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = 5f0b02c75b57c5855da9ae460ce51323ea669d8a
llama_model_loader: - kv   3:                           general.finetune str              = 5f0b02c75b57c5855da9ae460ce51323ea669d8a
llama_model_loader: - kv   4:                         general.size_label str              = 8.0B
llama_model_loader: - kv   5:                          llama.block_count u32              = 32
llama_model_loader: - kv   6:                       llama.context_length u32              = 8192
llama_model_loader: -

user

You are a helpful assistant. Reply in Traditional Chinese.assistant

( w�� huì bāng zhù nǐ )

> EOF by user




llama_perf_sampler_print:    sampling time =       0.79 ms /    36 runs   (    0.02 ms per token, 45801.53 tokens per second)
llama_perf_context_print:        load time =     938.65 ms
llama_perf_context_print: prompt eval time =     580.17 ms /    21 tokens (   27.63 ms per token,    36.20 tokens per second)
llama_perf_context_print:        eval time =    4207.30 ms /    14 runs   (  300.52 ms per token,     3.33 tokens per second)
llama_perf_context_print:       total time =    4791.82 ms /    35 tokens


* 在非互動模式下，會用 `-n` 指定生成多少 Tokens，並搭配 `--ignore-eas` 來評測模型或硬體的速度。
* 透過 `-c` 參數可以指定模型的輸入長度，預設只有 512，但目前的模型都支援到 4096 或 8192 以上了，所以如果有相關應用要消耗大量 Token 的話，這個參數記得一定要開大，不然模型的輸出會看起來很奇怪。
* 在純 CPU 運算時，參數 `--threads` 對速度的影響滿大的，建議開系統總核心數的一半，例如你是 20 核的 CPU，那就指定 `--threads 10` 這樣。
* 另外能透過 `-fa` 來啟用 Flash Attention 加速生成。
* 其他像是 `--top-k`、`--top-p`、`--temp` (也就是Temperature) 等都是老面孔的取樣參數了，除此之外 llama.cpp 還額外支援相當豐富的取樣參數，可以自行研究看看。
* 補充：
    * 其中把 `--temp` 設定為 0.0 並且把 `--repeat-penalty` 設定為 1.0，會最貼近在 HF Transformers 套件裡面設定 `do_sample=False` 的結果。